<a href="https://colab.research.google.com/github/aliasharf123/arabic-summarization-with-araBert/blob/main/arabic_summarization_with_araBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Summerization content With AraBert

## Install dependencies  

In [ ]:
! pip install sentencepiece

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
! pip install transformers
! pip install datasets
! pip install rouge_score
! pip install wandb

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=7a55d08b6311b97de90fe97f60663318b7e5d2dccd1cdfdab88af0e6f7a3ec82
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install rouge


In [ ]:
!pip install rouge_score

## Import libraries

In [ ]:
from datasets import Dataset
import torch
import numpy as np
import datasets
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
import pandas as pd
from rouge import Rouge
from prettytable import PrettyTable
from tabulate import tabulate
import nltk
from datetime import datetime

## Initilize the AraBert model from hugging face

In [ ]:
# Define a mode name
model_name="abdalrahmanshahrour/arabartsummarization"

# Get a pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Get it's Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

### Show a model configuration

In [ ]:
# Assuming model.config is a dictionary
model_config_dict = model.config.to_dict()

# Create a PrettyTable instance
table = PrettyTable()

# Add columns
table.field_names = ["Parameter", "Value"]

# Add rows
for parameter, value in model_config_dict.items():
    table.add_row([parameter, value])

# Print the table
print(table)

+----------------------------------+--------------------------------------------+
|            Parameter             |                   Value                    |
+----------------------------------+--------------------------------------------+
|            vocab_size            |                   50002                    |
|     max_position_embeddings      |                    1024                    |
|             d_model              |                    768                     |
|         encoder_ffn_dim          |                    3072                    |
|          encoder_layers          |                     6                      |
|     encoder_attention_heads      |                     12                     |
|         decoder_ffn_dim          |                    3072                    |
|          decoder_layers          |                     6                      |
|     decoder_attention_heads      |                     12                     |
|             dr

## Load arabic XL-Sum dataset in hugging face

#### Load dataset

In [ ]:
dataset = datasets.load_dataset("csebuetnlp/xlsum", name="arabic", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/arabic/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


### Show dataset

In [ ]:
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

# Display the dataset structure
print("Dataset Structure:")
print("------------------")
print(df.info())

# Display the first few rows of the DataFrame
num_rows_to_display = 5
print("\nFirst Few Rows of the Dataset:")
df.head()

Dataset Structure:
------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37519 entries, 0 to 37518
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       37519 non-null  object
 1   url      37519 non-null  object
 2   title    37519 non-null  object
 3   summary  37519 non-null  object
 4   text     37519 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB
None

First Few Rows of the Dataset:


,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


## Prepocess the Dataset


### Remove the not wanted columns

In [ ]:
dataset = dataset.remove_columns(["id","url","title"])

In [ ]:
dataset

Dataset({
    features: ['summary', 'text'],
    num_rows: 37519
})

### Split the dataset

In [ ]:
train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.3).values()

### Batch processing function for tokenization

In [ ]:
decoder_max_length = 128
encoder_max_length = 512

def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    #source, target = batch["document"], batch["summary"]
    source, target = batch["text"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    # Stripping leading and trailing whitespace
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # Tokenization using NLTK's sent_tokenize and joining with newline characters
    # RougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    # Extracting Predictions and Labels:
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # Decoding Predictions and Labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Use the ROUGE metric to compute evaluation results
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    # Extract specific ROUGE scores (mid.fmeasure) and convert them to percentages
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Count non-pad tokens in each prediction to compute average length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]

    # Compute the average length of the generated predictions
    result["gen_len"] = np.mean(prediction_lens)

    # Round all values in the result dictionary to four decimal places
    result = {k: round(v, 4) for k, v in result.items()}


    return result

## Train the model

### Initlize the trainer

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    seed=42,
    learning_rate=5e-05,
    warmup_steps=250,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
    lr_scheduler_type="linear",
    optim="adamw_hf",
    adam_beta1 = 0.9,
    adam_beta2= 0.999,
    adam_epsilon= 1e-08,
)

# Initialize data collator for sequence-to-sequence task
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

### login in wandb for training

In [ ]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Start train model

In [ ]:
#final
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: aliashrafali770 (aliashraf123). Use `wandb login --relogin` to force relogin


You're using a BarthezTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,4.051000
100,3.862700
150,3.793200
200,3.820200
250,3.844700
300,3.798300
350,3.861400
400,3.794700
450,3.798700
500,3.756000


TrainOutput(global_step=16415, training_loss=3.399114825236808, metrics={'train_runtime': 8719.933, 'train_samples_per_second': 15.059, 'train_steps_per_second': 1.882, 'total_flos': 4.003501075070976e+16, 'train_loss': 3.399114825236808, 'epoch': 5.0})

## Save the model

In [ ]:
model_dir = 'rtsumm/kaggle/working/arabaarization/'
trainer.save_model(model_dir + 'model2')

In [ ]:
# Save the fine-tuned model to Google Drive
model.save_pretrained("rtsumm/kaggle/working/arabaarization/arabartsummarization/pretrained")

## Evaluate the model

In [ ]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 3.4821231365203857,
 'eval_rouge1': 2.3202,
 'eval_rouge2': 0.0977,
 'eval_rougeL': 2.3288,
 'eval_rougeLsum': 2.3113,
 'eval_gen_len': 19.6799,
 'eval_runtime': 955.4586,
 'eval_samples_per_second': 11.781,
 'eval_steps_per_second': 1.473,
 'epoch': 5.0}

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        #test_samples["document"],
        #test_samples["text"],
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    # Move the input tensors to the device where the model is located
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)

    # Generate summaries using the model
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # Decode the generated output sequences into human-readable text
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return outputs, output_str

### Load a test dataset

In [ ]:
testDataset = datasets.load_dataset("csebuetnlp/xlsum", name="arabic", split="test")

df = pd.DataFrame(testDataset)
df.head()

,id,url,title,summary,text
0,130806_nidhal_hassan_trial,https://www.bbc.com/arabic/worldnews/2013/08/1...,نضال حسن يمثل أمام محكمة عسكرية أمريكية,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,نضال حسن واعترف نضال حسن، الذي يدافع عن نفسه، ...
1,160129_germany_asylum_seekers,https://www.bbc.com/arabic/worldnews/2016/01/1...,ألمانيا تسعى للحد من الهجرة بإعلان الجزائر وتو...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,130729_syria_homs_area_retaken,https://www.bbc.com/arabic/middleeast/2013/07/...,الخالدية : التليفزيون السوري يعلن استعادة الجي...,قال التليفزيون السوري إن قوات الحكومة استعادت ...,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا..."
3,140517_arsenal_fa_cup_winners,https://www.bbc.com/arabic/sports/2014/05/1405...,الارسنال يتوج بلقب كأس انكلترا,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,140722_iraq_minorities,https://www.bbc.com/arabic/middleeast/2014/07/...,العراق: الأقليات في سهل نينوى,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...


### Get a rouge score for 40 row from test dataset

In [ ]:
testSource = testDataset['text'][0:40]
targetSource = testDataset['summary'][0:40]

In [ ]:
summaries_after_tuning = generate_summary(testSource , model)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
def calculate_rouge_scores(reference_summaries, generated_summaries):
    rouge = Rouge()
    # Convert the input lists to strings
    reference_summaries = [str(summary) for summary in reference_summaries]
    generated_summaries = [str(summary) for summary in generated_summaries]

    scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)
    return scores

rouge_scores = calculate_rouge_scores(targetSource, summaries_after_tuning[1])
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge-1': {'r': 0.24741749073764413, 'p': 0.36122868553015625, 'f': 0.28819034162831875}, 'rouge-2': {'r': 0.11422120115460718, 'p': 0.171666076978577, 'f': 0.1343900924253511}, 'rouge-l': {'r': 0.22742818789489294, 'p': 0.33283126799670926, 'f': 0.2654852832561154}}


### Test with a text

In [ ]:
text = "شهدت مدينة طرابلس، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي والاقتصادي. واندلعت مواجهات عنيفة وعمليات كر وفر ما بين الجيش اللبناني والمحتجين استمرت لساعات، إثر محاولة فتح الطرقات المقطوعة، ما أدى إلى إصابة العشرات من الطرفين."
print("text before summerize: \n" , text)

result = generate_summary(text , model)
print("text after summerize: \n" , result[1])

text before summerize: 
 شهدت مدينة طرابلس، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي والاقتصادي. واندلعت مواجهات عنيفة وعمليات كر وفر ما بين الجيش اللبناني والمحتجين استمرت لساعات، إثر محاولة فتح الطرقات المقطوعة، ما أدى إلى إصابة العشرات من الطرفين.
text after summerize: 
 ['تجددت الاشتباكات في مدينة طرابلس اللبنانية بين الجيش اللبناني والمحتجين لليوم الثالث على التوالي']
